## PLS

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = 'data/' 
file = 'ST000369.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 163 TOTAL PEAKS: 180
Done!


### 3. Extract X & Y

In [4]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)    



### 4. Hyperparameters optimisation

In [6]:
# param_dict
param_dict = {'n_components': [1,2,3,4,5,6]}                   
#param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12]} 


# Initalise
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=100)                                

# Run and plot
cv.run()  
cv.plot(metric='r2q2', ci=95)

Number of cores set to: 8
Running ...


100%|██████████| 600/600 [00:08<00:00, 69.32it/s] 


Time taken: 0.15 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [8]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.PLS_SIMPLS(n_components=2)
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate Ypred against Ytest
model.evaluate(cutoffscore=0.5, plot_median=False) 

Loading BokehJS ...

In [10]:
# Bootstrap evaluate
model.booteval(XTknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:00<00:00, 159.93it/s]


Loading BokehJS ...

### 7. Save table

In [11]:
home = 'tables/'
file = 'PLS_ST000369_NH.xlsx'

model.save_table(home + file)

Done! Saved table as tables/PLS_ST000369_NH.xlsx
